In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("fr_wiki")

Vocabulary will not be loaded from file.



In [3]:
semiotic.vocab.build(vocab_size = 503,parallel=False, parallel_mode="process")


Alphabet Size: 498



Vocabulary built


In [4]:
chainlist_seq = semiotic.vocab.chain_list
alpha_seq = semiotic.vocab.alpha

In [5]:
semiotic.vocab.build(vocab_size = 503,parallel=True, parallel_mode="process")


Alphabet Size: 498



Vocabulary built


In [6]:
print(chainlist_seq == semiotic.vocab.chain_list)
print(alpha_seq == semiotic.vocab.alpha)

True
True


In [22]:
n = 455700
chainlist_seq[:n] == semiotic.vocab.chain_list[:n]

True

In [7]:
[(i,(l,r)) for i,(l,r) in enumerate(zip(sorted(alpha_seq.items()),sorted(semiotic.vocab.alpha.items()))) if l!=r]

[(1, (('1', 106323), ('1', 106327))),
 (2, (('2', 57157), ('2', 57158))),
 (3, (('3', 26276), ('3', 26278))),
 (6, (('6', 23762), ('6', 23763))),
 (9, (('9', 65768), ('9', 65773))),
 (10, (('a', 1407511), ('a', 1407529))),
 (12, (('c', 610267), ('c', 610278))),
 (13, (('d', 761112), ('d', 761124))),
 (14, (('e', 2519128), ('e', 2519164))),
 (15, (('f', 198180), ('f', 198184))),
 (16, (('g', 232012), ('g', 232014))),
 (18, (('i', 1307647), ('i', 1307653))),
 (19, (('j', 59196), ('j', 59199))),
 (21, (('l', 1048131), ('l', 1048140))),
 (22, (('m', 494139), ('m', 494146))),
 (23, (('n', 1309534), ('n', 1309561))),
 (24, (('o', 976389), ('o', 976402))),
 (25, (('p', 497908), ('p', 497912))),
 (26, (('q', 130512), ('q', 130514))),
 (27, (('r', 1231139), ('r', 1231159))),
 (28, (('s', 1335796), ('s', 1335809))),
 (29, (('t', 1204908), ('t', 1204916))),
 (30, (('u', 923685), ('u', 923701))),
 (31, (('v', 224262), ('v', 224263))),
 (33, (('x', 77466), ('x', 77467))),
 (35, (('z', 25807), ('z',

In [ ]:
[(i,(l,r)) for i,(l,r) in enumerate(zip(chainlist_seq,semiotic.vocab.chain_list)) if l!=r]

In [4]:
def parallel_chain(chain, n_of_parts):
    """
    Breaks the chain in n chunks to compute best pair of terms. Chunks are overlapping by one term, so as no pair of terms is lost due to the break.
    """
    if not isinstance(chain,list):
        chain = list(chain)
    chunk_size = int(len(chain) / n_of_parts)+1
    for i in range(0, len(chain), chunk_size):
        yield chain[i : i + chunk_size +1]

In [7]:
from temp import bla2
        
listtup = ["a","b","c","b","b","c","c","b","c","h","p"]

list(parallel_chain(listtup,4))

# result = slg.util.multiprocessing(bla2, listtup)


[['a', 'b', 'c', 'b'], ['b', 'b', 'c', 'c'], ['c', 'b', 'c', 'h'], ['h', 'p']]

In [3]:
result

[(0, ['a', 'b', 'c'], ['a', 'b', 'c']),
 (1, ['b', 'b', 'c'], ['b', 'b', 'c']),
 (2, ['c', 'b', 'c'], ['c', 'b', 'c'])]

In [3]:
corpus = semiotic.corpus.train

normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.vocab.config.normalizer)

chain_list = []

alphabet = Counter()

for sent in tqdm(corpus, desc="Chain List & Alphabet:"):
    sent = normalizer.normalize(sent)
    sent = list(sent)
    if sent !=[]:
        chain_list += sent
        alphabet.update(Counter(sent))

In [4]:
reload(temp)

<module 'temp' from '/Users/Gianni/semiolog/wip/temp.py'>

In [5]:
for i in range(5):
    now = datetime.now()
    result = util.multiprocessing(temp.find_best_pair, temp.parallel_chain(chain_list,4), cores = 4)

    parallel = reduce(operator.add, result)
        
    parallel = parallel.most_common()
    print(datetime.now() - now)

0:00:02.978095
0:00:02.831269
0:00:03.196081
0:00:03.024566
0:00:02.903223


In [25]:
for i in range(5):
    now = datetime.now()
    
    sequential = temp.find_best_pair(chain_list)
    sequential = sequential.most_common()
    
    print(datetime.now() - now)

0:00:05.527317
0:00:05.145561
0:00:05.272142
0:00:05.325645
0:00:05.272945


In [19]:
 parallel == sequential

True

In [3]:
semiotic.vocab.build(vocab_size = 1627)


Vocabulary built


In [4]:
semiotic.vocab.build(vocab_size = 1627, parallel=True, parallel_mode="process")


Vocabulary built


In [3]:
def chain_list_alpha(self, normalizer, corpus_sents, progress_bar = False):
    chain_list = []
    alphabet = Counter()
    for sent in corpus_sents:
        sent = normalizer.normalize(sent)
        sent = list(sent)
        if sent !=[]:
            chain_list += sent
            alphabet.update(Counter(sent))
    return chain_list, alphabet

4

In [3]:
from tqdm.notebook import tqdm
import time
from temp import chain_list_i, chain_list
from functools import partial

In [4]:
def parallel_chain(chain, n_of_parts):
    """
    Breaks the chain in n chunks to compute best pair of terms. Chunks are overlapping by one term, so as no pair of terms is lost due to the break.
    """
    if not isinstance(chain,list):
        chain = list(chain)
    chunk_size = int(len(chain) / n_of_parts)+1
    for i in range(0, len(chain), chunk_size):
        yield chain[i : i + chunk_size +1]

In [5]:
normalizer = eval(
                f"slg.syntagmatic.tokenizer.normalizers.Sequence({slg.util.if_none_disable(semiotic.config.vocabulary.normalizer)})"
            )

In [6]:
bla = chain_list(normalizer,semiotic.corpus.train)

In [ ]:
%%time

bla_p = slg.util.multiprocessing(partial(chain_list, normalizer),parallel_chain(semiotic.corpus.train,4),cores=4)

CPU times: user 303 ms, sys: 190 ms, total: 493 ms
Wall time: 5.57 s


In [ ]:
%%time

bla_t = slg.util.multithreading(partial(chain_list_i, normalizer),semiotic.corpus.train,cores=4)